<h1>Chapter 14. Inheritance: for better or for worse.</h1>

In Python, inheritance is a mechanism where a new class, called a subclass, is created based on an existing class, called a superclass. The subclass inherits attributes and behaviors (methods) from the superclass, allowing for code reusability and the creation of more complex systems by building on existing functionality. It supports single, multiple, and multilevel inheritance, facilitating a hierarchical organization of classes. This approach helps in reducing redundancy and enhancing the maintainability of the code by promoting a natural hierarchy and encapsulation of functionalities.

<h2><code>super()</code> Function</h2>

The `super()` function is used to give access to methods and properties of a parent or sibling class. It is typically used in a subclass to call a method from the parent class, enabling the subclass to inherit and extend the behavior of the parent class without explicitly naming it. This helps in maintaining the code, especially when dealing with multiple inheritance, as it allows the subclass to invoke methods in a way that respects the method resolution order (MRO).

In [1]:
from collections import OrderedDict


class LastUpdatedOrderedDict(OrderedDict):
    """Items are stored in the order determined by the last update"""

    def __setitem__(self, key, value):
        # Use the construct to call a superclass method
        super().__setitem__(key, value)
        self.__move_to_end(key, value)

<h2>Difficulties of Inheritance to Built-In Types</h2>

The direct inheritance of built-in types, such as `dict`, `list`, or `str`, is a practice that is susceptible to error due to the tendency of built-in methods to ignore user-defined overridden methods.

In [2]:
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

In [3]:
dd = DoppelDict(one=1)
dd

{'one': 1}

The `__init__` method inherited from dict apparently does not know that `__setitem__` is overridden: the value of `'one'` is not repeated.

In [4]:
dd['two'] = 2
dd

{'one': 1, 'two': [2, 2]}

The `[]` operator calls the `__setitem__` method and works as expected: `'two'` is mapped to the repeated value `[2, 2]`.

The `__getitem__` method from the `AnswerDict` class is ignored by the `dict.update` method.

In [5]:
class AnswerDict(dict):
    def __getitem__(self, key):
        return 42

In [6]:
ad = AnswerDict(a='foo')
ad['a']

42

In [7]:
d = {}
d.update(ad)
d['a']

'foo'

In [8]:
d

{'a': 'foo'}

Instead of creating subclasses of built-in objects, classes should be inherited from those in the `collections` module, such as `UserDict`, `UserList`, and `UserString`, which are specifically designed for seamless inheritance.

In [9]:
from collections import UserDict


class DoppelDict2(UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

In [10]:
dd2 = DoppelDict2(one=1)
dd2

{'one': [1, 1]}

In [11]:
dd2['two'] = 2
dd2

{'one': [1, 1], 'two': [2, 2]}

In [12]:
dd2.update(three=3)
dd2

{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}

In [13]:
class AnswerDict2(UserDict):
    def __getitem__(self, key):
        return 42

In [14]:
ad2 = AnswerDict2(a='foo')
ad2

{'a': 'foo'}

In [15]:
d = {}
ad2.update(d)
ad2['a']

42

In [16]:
ad2

{'a': 'foo'}

<h2>Multiple Inheritance and Method Resolution Procedure</h2>

In [17]:
class Root:
    """
    The Root class provides ping(), pong(), and __repr__ methods
    to make the output easier to read.
    """

    def ping(self):
        print(f"{self}.ping() in class Root.")

    def pong(self):
        print(f"{self}.pong() in class Root.")

    def __repr__(self):
        cls_name = type(self).__name__
        return f"<instance of {cls_name}>"


class A(Root):
    """Both the ping() and pong() methods in class A call super()"""

    def ping(self):
        print(f"{self}.ping() in class A.")
        super().ping()

    def pong(self):
        print(f"{self}.pong() in class A.")
        super().pong()


class B(Root):
    """In class B, only the ping() method calls super()"""

    def ping(self):
        print(f"{self}.ping() in class B.")
        super().ping()

    def pong(self):
        print(f"{self}.pong() in class B.")


class Leaf(A, B):
    """The Leaf class only implements the ping() method, which calls super()."""

    def ping(self):
        print(f"{self}.pong() in class Leaf.")
        super().ping()

In [18]:
leaf1 = Leaf()

Calling `leaf1.ping()` activates the `ping()` methods in `Leaf`, `A`, `B`, and `Root` because the `ping()` methods in the first three classes call `super().ping()`.

In [19]:
leaf1.ping()

<instance of Leaf>.pong() in class Leaf.
<instance of Leaf>.ping() in class A.
<instance of Leaf>.ping() in class B.
<instance of Leaf>.ping() in class Root.


A call to `leaf1.pong()` activates the `pong()` method in `A` due to inheritance, and that calls `super().pong()`, which activates `B.pong()`.

In [20]:
leaf1.pong()

<instance of Leaf>.pong() in class A.
<instance of Leaf>.pong() in class B.


Calling the `__mro__` attribute of the `Leaf` class, which stores a tuple of superclass references in the order of method resolution, from the current class to the object class.

In [21]:
Leaf.__mro__

(__main__.Leaf, __main__.A, __main__.B, __main__.Root, object)

Classes demonstarating the dynamic nature of `super()`

In [22]:
class U:
    """Class U is not related to either A or Root"""

    def ping(self):
        print(f"{self}.ping() in U class.")
        super().ping()


class LeafUA(U, A):
    """LeafUA inherits U and A in exactly that order."""

    def ping(self):
        print(f"{self}.ping() in LeafUA class.")
        super().ping()

In [23]:
try:
    u = U()
    u.ping()
except AttributeError as e:
    print(e.__repr__())

<__main__.U object at 0x109c33ec0>.ping() in U class.
AttributeError("'super' object has no attribute 'ping'")


In [24]:
leaf2 = LeafUA()
leaf2.ping()

<instance of LeafUA>.ping() in LeafUA class.
<instance of LeafUA>.ping() in U class.
<instance of LeafUA>.ping() in class A.
<instance of LeafUA>.ping() in class Root.


In [25]:
LeafUA.__mro__

(__main__.LeafUA, __main__.U, __main__.A, __main__.Root, object)

<h2>Classes - Mixins</h2>

A mixin class is intended to be inherited along with at least one other class when organizing multiple inheritance. A mixin should not be the sole base class of a particular class because it does not provide the full functionality of a specific object; instead, it only adds to or customizes the behavior of descendant or sibling classes.

<h3>Case - Insensitive Displays</h3>

`UpperCaseMixin` supports case-insensitive mappings

In [26]:
import collections


def _upper(key):
    try:
        return key.upper()
    except AttributeError:
        return key


class UpperCaseMixin:
    def __setitem__(self, key, item):
        super().__setitem__(_upper(key), item)

    def __getitem__(self, key):
        return super().__getitem__(_upper(key))

    def get(self, key, default=None):
        return super().get(_upper(key), default)

    def __contains__(self, key):
        return super().__contains__(_upper(key))

Two classes using `UpperCaseMixin`

In [27]:
class UpperDict(UpperCaseMixin, collections.UserDict):
    pass


class UpperCounter(UpperCaseMixin, collections.Counter):
    """A specialized 'Counter' that translates string keys to upper case."""

In [28]:
d = UpperDict([('a', 'letter A'), (2, 'digit two')])
list(d.keys())

['A', 2]

In [29]:
d['b'] = 'letter B'
'b' in d

True

In [30]:
d['a'], d.get('B')

('letter A', 'letter B')

In [31]:
list(d.keys())

['A', 2, 'B']

In [32]:
c = UpperCounter('BaNanA')
c.most_common()

[('A', 3), ('N', 2), ('B', 1)]